In [2]:
# conda install -c conda-forge/label/gcc7 folium

In [398]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fast')

from sklearn.pairwise import cosine_similarity
from src.utilities import *
from src.matrix_factorization_utilities import *

ModuleNotFoundError: No module named 'sklearn.pairwise'

# Datasets

In [4]:
ratings_df = csv('https://markg110.s3-us-west-1.amazonaws.com/data/BX-Book-Ratings.csv')
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
ratings_df.shape

(1149780, 3)

In [6]:
ratings_df.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [8]:
books_df = csv('https://markg110.s3-us-west-1.amazonaws.com/data/BX-Books.csv')
books_df.head()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
users_df = csv('https://markg110.s3-us-west-1.amazonaws.com/data/BX-Users.csv')
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


# Similarity Recommender

In [102]:
print('Ratings Shape: ', ratings_df.shape)
print('Users Shape: ', users_df.shape)
print('Book Shape: ', books_df.shape)

Ratings Shape:  (1149780, 3)
Users Shape:  (278858, 3)
Book Shape:  (271360, 8)


In [78]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [79]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [132]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",nan
1,2,"stockton, california, usa",18.000
2,3,"moscow, yukon territory, russia",nan
3,4,"porto, v.n.gaia, portugal",17.000
4,5,"farnborough, hants, united kingdom",nan


---
## Correlational Recommender

---

In [133]:
# Rating Count
rating_count = pd.DataFrame(ratings_df.groupby('ISBN')['Book-Rating'].count())

In [137]:
rating_count.sort_values('Book-Rating', ascending=False).head()

,Book-Rating
ISBN,
0971880107,2502
0316666343,1295
0385504209,883
0060928336,732
0312195516,723


In [138]:
most_rated_books = pd.DataFrame(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516'], index=np.arange(5), columns=['ISBN'])
most_rated_books

,ISBN
0,0971880107
1,0316666343
2,0385504209
3,0060928336
4,0312195516


In [139]:
most_rated_books_summary = pd.merge(most_rated_books, books_df, on='ISBN')
most_rated_books_summary

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0971880107,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...
1,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...
2,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...
3,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...
4,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...


## Recommendations Based on Correlations

In [140]:
average_rating = pd.DataFrame(ratings_df.groupby('Publisher')['Book-Rating'].mean()) # <- mean()
average_rating.head()

,Book-Rating
ISBN,
0330299891,3.000
0375404120,1.500
0586045007,0.000
9022906116,3.500
9032803328,0.000


In [141]:
average_rating['Rating-Count'] = pd.DataFrame(ratings_df.groupby('ISBN')['Book-Rating'].count()) # <- count()
average_rating.sort_values('Rating-Count', ascending=False).head()

,Book-Rating,Rating-Count
ISBN,,
0971880107,1.020,2502
0316666343,4.469,1295
0385504209,4.652,883
0060928336,3.448,732
0312195516,4.335,723


In [172]:
average_rating['Rating-Count'].describe()

count   340556.000
mean         3.376
std         12.436
min          1.000
25%          1.000
50%          1.000
75%          2.000
max       2502.000
Name: Rating-Count, dtype: float64

In [165]:
select_year_df = books_df[books_df['Year-Of-Publication'] != 'Gallimard']

In [166]:
select_year_df = select_year_df[select_year_df['Year-Of-Publication'] != 'DK Publishing Inc']

In [167]:
select_year_df['Year-Of-Publication'] = select_year_df['Year-Of-Publication'].apply(lambda x: int(x))

In [176]:
publisher_count = select_year_df['Publisher'].value_counts().to_frame().reset_index().rename(columns={'index':'Publisher','Publisher':'Count'})
columns=['Publisher','Image-URL-S', 'Image-URL-M', 'Image-URL-L']
select_year_df.drop(columns,1,inplace=True)
select_year_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999


In [190]:
publisher_count = publisher_count[:30]

In [228]:
select_year_df.rename(columns={'Year-Of-Publication':'Publication Year'},inplace=True)

In [230]:
columns = ['Book-Author']
select_year_df.drop(columns=columns,inplace=True)
select_year_df.head()

,ISBN,Book-Title,Publication Year
0,0195153448,Classical Mythology,2002
1,0002005018,Clara Callan,2001
2,0060973129,Decision in Normandy,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,1999
4,0393045218,The Mummies of Urumchi,1999


In [231]:
average_rating.head()

,Book-Rating,Rating-Count
ISBN,,
0330299891,3.000,2
0375404120,1.500,2
0586045007,0.000,1
9022906116,3.500,2
9032803328,0.000,1


In [232]:
average_rating.shape

(340556, 2)

In [233]:
select_year_df.shape

(271357, 3)

In [271]:
year_bound = pd.merge(average_rating, select_year_df, on='ISBN')
year_bound.head()

,ISBN,Book-Rating,Rating-Count,Book-Title,Publication Year
0,0000913154,8.000,1,The Way Things Work: An Illustrated Encycloped...,1967
1,0001010565,0.000,2,Mog's Christmas,1992
2,0001046438,9.000,1,Liar,0
3,0001046713,0.000,1,Twopence to Cross the Mersey,1992
4,000104687X,6.000,1,"T.S. Eliot Reading \The Wasteland\"" and Other ...",1993


In [272]:
year_bound.shape

(270148, 5)

In [273]:
# Trimming down dataset by publication year: 1975 to 2002
year_bound = year_bound[year_bound['Publication Year'] >= 1975]
year_bound = year_bound[year_bound['Publication Year'] <= 2002]
year_bound.shape

(239066, 5)

In [274]:
year_bound[year_bound['Rating-Count'] < 100].describe()

,Book-Rating,Rating-Count,Publication Year
count,238423.000,238423.000,238423.000
mean,2.821,3.282,1993.582
std,3.262,6.702,6.646
min,0.000,1.000,1975.000
25%,0.000,1.000,1989.000
50%,1.667,1.000,1995.000
75%,5.000,3.000,1999.000
max,10.000,99.000,2002.000


In [275]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [276]:
columns=['Book-Rating']
ratings = ratings_df.drop(columns,1)

In [277]:
year_bound_combine = pd.merge(year_bound, ratings, on='ISBN')

In [278]:
year_bound_combine.head()

,ISBN,Book-Rating,Rating-Count,Book-Title,Publication Year,User-ID
0,0001010565,0.000,2,Mog's Christmas,1992,86123
1,0001010565,0.000,2,Mog's Christmas,1992,209516
2,0001046713,0.000,1,Twopence to Cross the Mersey,1992,196149
3,000104687X,6.000,1,"T.S. Eliot Reading \The Wasteland\"" and Other ...",1993,23902
4,0001046934,0.000,1,The Prime of Miss Jean Brodie,1999,206300


In [281]:
year_bound_combine.shape

(902959, 6)

In [280]:
year_bound_combine['Publication Year'].describe()

count   902959.000
mean      1994.936
std          6.036
min       1975.000
25%       1991.000
50%       1996.000
75%       2000.000
max       2002.000
Name: Publication Year, dtype: float64

In [283]:
year_bound_combine['User-ID'].value_counts().describe()

count   84457.000
mean       10.691
std        81.135
min         1.000
25%         1.000
50%         1.000
75%         4.000
max      9484.000
Name: User-ID, dtype: float64

In [290]:
# Trimming dataset to users with at least 200 ratings given and books that were rated 100 times or more
counts = year_bound_combine['User-ID'].value_counts()
counts1 = year_bound_combine['ISBN'].value_counts()
iso_df = year_bound_combine[year_bound_combine['User-ID'].isin(counts[counts >= 200].index)]
iso_df = year_bound_combine[year_bound_combine['ISBN'].isin(counts1[counts1 >= 100].index)]

In [356]:
print(iso_df.shape)
print('')
print(iso_df['ISBN'].value_counts().describe())
print('')
print(iso_df['ISBN'].value_counts().head())
print('')
iso_df.sample(10)

(120358, 6)

count    643.000
mean     187.182
std      109.660
min      100.000
25%      119.000
50%      150.000
75%      211.500
max     1295.000
Name: ISBN, dtype: float64

0316666343    1295
0060928336     732
0312195516     723
044023722X     647
067976402X     614
Name: ISBN, dtype: int64



,ISBN,Book-Rating,Rating-Count,Book-Title,Publication Year,User-ID
236779,0380718332,2.750,184,All That Remains (Kay Scarpetta Mysteries (Pap...,1993,114414
261286,0385265700,2.668,319,The Book of Ruth (Oprah's Book Club (Paperback)),1990,34539
402618,0446611867,3.264,250,A Bend in the Road,2002,204522
510235,0553275976,2.143,189,The Bonfire of the Vanities,1988,224764
512057,0553279378,3.137,175,I Know Why the Caged Bird Sings,1983,117912
574421,0671004549,2.732,123,You Belong To Me,1999,204804
399485,0446608955,3.430,284,A Walk to Remember,2000,236283
636678,0679744398,2.231,147,"All the Pretty Horses (The Border Trilogy, Vol 1)",1993,20119
159762,0345384466,3.177,243,The Witching Hour (Lives of the Mayfair Witches),1993,99032
582216,0671027387,2.862,247,Deception Point,2002,251164


## Rating Matrix

In [300]:
iso_df_pivot = pd.pivot_table(iso_df, index='User-ID', columns='ISBN', values='Book-Rating')
rating_matrix = iso_df_pivot.copy()
print('')
print("Shape of Rating Matrix:",rating_matrix.shape)
print('')
rating_matrix.head()


Shape of Rating Matrix: (29416, 643)



ISBN,002542730X,0060096195,006016848X,0060173289,0060175400,006019491X,0060199652,0060391626,0060392452,0060502258,...,1558743669,1558744150,1558744630,1558745157,1573225517,1573225789,1573227331,1573229326,1573229571,1878424319
User-ID,,,,,,,,,,,,,,,,,,,,,
9,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
16,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
26,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
39,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
42,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [328]:
print('')
print('')
print("Shape prior to pivoting:",iso_df.shape)
print('')
iso_df.sort_values(by='Book-Rating').sample(5)



Shape prior to pivoting: (120358, 6)



,ISBN,Book-Rating,Rating-Count,Book-Title,Publication Year,User-ID
348327,0440219078,4.916,178,The Giver (21st Century Reference),1994,65032
459367,0452283205,3.870,131,Falling Angels,2002,3191
431182,0451167317,3.392,153,The Dark Half,1994,24921
361903,044023722X,3.187,647,A Painted House,2001,249930
395251,0446605239,3.406,465,The Notebook,1998,221103


## Begin Recommendation: Pearson Correlation

In [338]:
iso_df['ISBN'].describe()

count         120358
unique           643
top       0316666343
freq            1295
Name: ISBN, dtype: object

In [354]:
print(average_rating.shape)
print('')
print(average_rating.head())

(340556, 2)

             Book-Rating  Rating-Count
ISBN                                  
 0330299891        3.000             2
 0375404120        1.500             2
 0586045007        0.000             1
 9022906116        3.500             2
 9032803328        0.000             1


In [362]:
user_id = rating_matrix.index 
book_isbn = rating_matrix.columns

In [376]:
print(average_rating.shape)
print('')
average_rating.sort_values('Rating-Count', ascending=False).head()

(340556, 2)



,Book-Rating,Rating-Count
ISBN,,
0971880107,1.020,2502
0316666343,4.469,1295
0385504209,4.652,883
0060928336,3.448,732
0312195516,4.335,723


In [377]:
isbn_lookup = rating_matrix['0312195516']
similar_to_isbn = rating_matrix.corrwith(isbn_lookup)
correlation = pd.DataFrame(similar_to_isbn, columns=['pearson R'])
correlation.dropna(inplace=True)
correlation_summary = correlation.join(average_rating['Rating-Count'])

In [ ]:
# recommendation_pearson = correlation_summary[correlation_summary['Rating-Count'] >= 300].sort_values('pearson R', ascending=False).head(10)
# recommendation_pearson

In [396]:
recommendation_pearson = correlation_summary.sort_values('pearson R', ascending=False).head(10)
recommendation_pearson.sort_values('Rating-Count', ascending=False)

,pearson R,Rating-Count
ISBN,,
0440236673,1.000,408
0452282829,1.000,342
0449005615,1.000,246
0451166892,1.000,228
0452264464,1.000,180
0451169530,1.000,131
0452282195,1.000,122
0446675059,1.000,122
0060096195,1.000,107


In [397]:
recommendation_titles_R = pd.merge(recommendation, books_df, on='ISBN')
recommendation_titles_R

,ISBN,pearson R,Rating-Count,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0060096195,1.000,107,The Boy Next Door,Meggin Cabot,2002,Avon Trade,http://images.amazon.com/images/P/0060096195.0...,http://images.amazon.com/images/P/0060096195.0...,http://images.amazon.com/images/P/0060096195.0...
1,0440236673,1.000,408,The Brethren,John Grisham,2000,Island,http://images.amazon.com/images/P/0440236673.0...,http://images.amazon.com/images/P/0440236673.0...,http://images.amazon.com/images/P/0440236673.0...
2,0452282829,1.000,342,We Were the Mulvaneys,Joyce Carol Oates,1996,Plume,http://images.amazon.com/images/P/0452282829.0...,http://images.amazon.com/images/P/0452282829.0...,http://images.amazon.com/images/P/0452282829.0...
3,0452282195,1.000,122,The Bluest Eye,Toni Morrison,2000,Plume Books,http://images.amazon.com/images/P/0452282195.0...,http://images.amazon.com/images/P/0452282195.0...,http://images.amazon.com/images/P/0452282195.0...
4,0452264464,1.000,180,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...
5,0451197747,1.000,102,The Deep End of the Ocean,Jacquelyn Mitchard,1999,Signet Book,http://images.amazon.com/images/P/0451197747.0...,http://images.amazon.com/images/P/0451197747.0...,http://images.amazon.com/images/P/0451197747.0...
6,0451169530,1.000,131,The Stand: Complete and Uncut,Stephen King,1991,Signet Book,http://images.amazon.com/images/P/0451169530.0...,http://images.amazon.com/images/P/0451169530.0...,http://images.amazon.com/images/P/0451169530.0...
7,0451166892,1.000,228,The Pillars of the Earth,Ken Follett,1996,Signet Book,http://images.amazon.com/images/P/0451166892.0...,http://images.amazon.com/images/P/0451166892.0...,http://images.amazon.com/images/P/0451166892.0...
8,0449005615,1.000,246,Seabiscuit: An American Legend,LAURA HILLENBRAND,2002,Ballantine Books,http://images.amazon.com/images/P/0449005615.0...,http://images.amazon.com/images/P/0449005615.0...,http://images.amazon.com/images/P/0449005615.0...
9,0446675059,1.000,122,The Honk and Holler Opening Soon,Billie Letts,1999,Warner Books,http://images.amazon.com/images/P/0446675059.0...,http://images.amazon.com/images/P/0446675059.0...,http://images.amazon.com/images/P/0446675059.0...


## Begin Recommendation: Cosine Similarity

## NMF - Non-negative Low Rank Matrix Factorization

In [10]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [395]:
ratings_df.shape

(1149780, 3)

In [12]:
ratings_df['User-ID'].head()

0    276725
1    276726
2    276727
3    276729
4    276729
Name: User-ID, dtype: int64

In [56]:
ratings_df.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [394]:
ratings_df.shape

(1149780, 3)

In [45]:
# Reducing rows to meet memory requirements. Original shape: (1149780 rows × 3 columns)
# resized_df = ratings_df[:200000]

In [69]:
# resized_df.head()

In [70]:
# resized_df.shape

In [66]:
# resized_df['User-ID'].value_counts().to_frame().sort_values(by='User-ID', ascending=True).head(20)

In [49]:
# pivot_ratings = pd.pivot_table(resized_df, index='User-ID', columns='ISBN', values='Book-Rating', aggfunc=np.max)

In [67]:
# pivot_ratings.head()

In [68]:
# pivot_ratings.shape

In [65]:
# U, B = low_rank_matrix_factorization(pivot_ratings.values, num_features=15, regularization_amount=1.0)